In [6]:
! pip install numpy pickle-mixin pandas tensorflow==1.13.2 keras  scikit-learn
import pathlib
import os
import pickle
import tensorflow as tf
import numpy as np
import sys
from random import randint
import datetime
from sklearn.utils import shuffle

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
# with open('/content/drive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# drive.flush_and_unmount()
# print("All don beero")
path_to_Shiba = '/content/drive/My Drive/Colab Notebooks/Shiba'

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


CHAT DB Creation

In [2]:
import sqlite3
import json
from datetime import datetime
import time


datarange = ['2007-03','2015-01']
for timeframe in datarange:
    print("timefram running",timeframe)
    sql_transaction = []
    start_row = 0
    cleanup = 1000000
    connection = sqlite3.connect(path_to_Shiba+'/chatdata/{}.db'.format(timeframe))
    c = connection.cursor()

    def create_table():
        c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

    def format_data(data):
        data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
        return data

    def transaction_bldr(sql):
        global sql_transaction
        sql_transaction.append(sql)
        if len(sql_transaction) > 1000:
            c.execute('BEGIN TRANSACTION')
            for s in sql_transaction:
                try:
                    c.execute(s)
                except:
                    pass
            connection.commit()
            sql_transaction = []

    def sql_insert_replace_comment(commentid ,parentid ,parent ,comment ,subreddit ,time ,score):
        try:
            sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
            transaction_bldr(sql)
        except Exception as e:
            print('replace insertion',str(e))

    def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
        try:
            sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
            transaction_bldr(sql)
        except Exception as e:
            print('has parent insertion',str(e))

    def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
        try:
            sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
            transaction_bldr(sql)
        except Exception as e:
            print('no parent insertion',str(e))



    def acceptable(data):
        if len(data.split(' ')) > 1000 or len(data) < 1:
            return False
        elif len(data) > 32000:
            return False
        elif data == '[deleted]':
            return False
        elif data == '[removed]':
            return False
        else:
            return True

    def find_parent(pid):
        try:
            sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
            c.execute(sql)
            result = c.fetchone()
            if result != None:
                return result[0]
            else: return False
        except Exception as e:
            #print(str(e))
            return False

    def find_existing_score(pid):
        try:
            sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
            c.execute(sql)
            result = c.fetchone()
            if result != None:
                return result[0]
            else: return False
        except Exception as e:
            #print(str(e))
            return False          


    if __name__== "__main__" :
        create_table()
        row_counter=0
        paired_rows=0

    with open(path_to_Shiba+"/chatdata/{}/RC_{}".format(timeframe.split('-')[0],timeframe),buffering=10000) as f:
        for row in f:
            #print(row)
            row_counter += 1
            row = json.loads(row)
            parent_id = row['parent_id']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            if timeframe.split('-')[0] == '2015':
                comment_id = row['name']
            else :
                comment_id = "t1_"+ row['id']
            subreddit = row['subreddit']
            parent_data = find_parent(parent_id)
            if score >= 2:
                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        if acceptable(body):
                            sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)

                else:
                    if acceptable(body):
                        if parent_data:
                            sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                            paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)

            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))



timefram running 2007-03
Total Rows Read: 100000, Paired Rows: 0, Time: 2021-05-01 10:25:26.648467
timefram running 2015-01
Total Rows Read: 100000, Paired Rows: 0, Time: 2021-05-01 10:26:51.039473
Total Rows Read: 200000, Paired Rows: 0, Time: 2021-05-01 10:28:07.592377
Total Rows Read: 300000, Paired Rows: 0, Time: 2021-05-01 10:29:23.165569


KeyboardInterrupt: ignored

CHAT DATA creation for training and testing purpose from DB data

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import pickle
from collections import Counter

datarange = ['2007-03','2015-01']
for timeframe in datarange:
    connection = sqlite3.connect(path_to_Shiba+'/chatdata/{}.db'.format(timeframe))
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False
    while cur_length == limit :
        dataframe = pd.read_sql("SELECT * from parent_reply where unix > {} AND parent NOT NULL AND score > 2 ORDER BY unix ASC LIMIT {}".format(last_unix,limit),connection)
        last_unix = dataframe.tail(1)['unix'].values[0]
        cur_length =  len(dataframe)
        wordliststr=""
        if not test_done:
            with open(path_to_Shiba+"/chatdata/{}test.from".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['parent'].values:
                    f.write(content+'\n')
                    wordliststr +=content

            with open(path_to_Shiba+"/chatdata/{}test.to".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['comment'].values:
                    f.write(content+'\n')
                    wordliststr +=content

            #for creating a test file in which parent and reply are in sequential order      
            comList = dataframe['parent'].values
            repList = dataframe['comment'].values
            logdict={}
            for i in range(len(comList)):
                logdict[comList[i]] = repList[i]
            np.save( path_to_Shiba+"/chatdata/comrepTest{}.npy".format(timeframe), logdict)
            
            wordlistdict = Counter(wordliststr.split())
            wordlist = list(wordlistdict.keys())
            with open(path_to_Shiba+"/wordlist.txt","wb")as fp:
                pickle.dump(wordlist,fp)
            
            test_done = True
            
        else:
            # training files in which from contains the comment and to contain the correspinding reply at corresponding line number
            with open(path_to_Shiba+"/chatdata/{}train.from".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['parent'].values:
                    f.write(content+'\n')
                    wordliststr += content

            with open(path_to_Shiba+"/chatdata/{}train.to".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['comment'].values:
                    f.write(content+'\n')
                    wordliststr += content

              #for creating a training file in which parent and reply are in sequential order      
            comList = dataframe['parent'].values
            repList = dataframe['comment'].values
            logdict={}
            for i in range(len(comList)):
                logdict[comList[i]] = repList[i]
            np.save( path_to_Shiba+"/chatdata/comrepTrain{}.npy".format(timeframe), logdict)
            
            wordlistdict = Counter(wordliststr.split())
            wordlist = list(wordlistdict.keys())
            with open(path_to_Shiba+"/wordlist.txt","wb")as fp:
                pickle.dump(wordlist,fp)
            test_done = True 

        
        counter+=1
        
        if counter%20 == 0 :
            print(counter*limit,'rows completed so far')
        

MODEL CREATION


In [3]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
#import tensorflow_addons as tfa
import numpy as np
import sys
from random import randint
import datetime
from sklearn.utils import shuffle
import pickle
import os

# Removes an annoying Tensorflow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

def createTrainMatrix(conversationFileName, wList, maxLen):
    conversationDictionary = np.load(conversationFileName,allow_pickle=True).item()
    numExamples = len(conversationDictionary)
    xTrain = np.zeros((numExamples, maxLen), dtype='int32')
    yTrain = np.zeros((numExamples, maxLen), dtype='int32')
    for index,(key,value) in enumerate(conversationDictionary.items()):
        # Will store integerized representation of strings here (initialized as padding)
        encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
        decoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
        # Getting all the individual words in the strings
        keySplit = key.split()
        valueSplit = value.split()
        keyCount = len(keySplit)
        valueCount = len(valueSplit)
        # Throw out sequences that are too long or are empty
        if (keyCount > (maxLen - 1) or valueCount > (maxLen - 1) or valueCount == 0 or keyCount == 0):
            continue
        # Integerize the encoder string
        for keyIndex, word in enumerate(keySplit):
            try:
                encoderMessage[keyIndex] = wList.index(word)
            except ValueError:
                # TODO: This isnt really the right way to handle this scenario
                encoderMessage[keyIndex] = 0
        encoderMessage[keyIndex + 1] = wList.index('<EOS>')
        # Integerize the decoder string
        for valueIndex, word in enumerate(valueSplit):
            try:
                decoderMessage[valueIndex] = wList.index(word)
            except ValueError:
                decoderMessage[valueIndex] = 0
        decoderMessage[valueIndex + 1] = wList.index('<EOS>')
        xTrain[index] = encoderMessage
        yTrain[index] = decoderMessage
    # Remove rows with all zeros
    yTrain = yTrain[~np.all(yTrain == 0, axis=1)]
    xTrain = xTrain[~np.all(xTrain == 0, axis=1)]
    numExamples = xTrain.shape[0]
    return numExamples, xTrain, yTrain

        

    
    

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [8]:
def getTrainingBatch(localXTrain, localYTrain, localBatchSize, maxLen):
    num = randint(0,numTrainingExamples - localBatchSize - 1)
    arr = localXTrain[num:num + localBatchSize]
    labels = localYTrain[num:num + localBatchSize]
    # Reversing the order of encoder string apparently helps as per 2014 paper
    reversedList = list(arr)
    for index,example in enumerate(reversedList):
        reversedList[index] = list(reversed(example))

    # Lagged labels are for the training input into the decoder
    laggedLabels = []
    EOStokenIndex = wordList.index('<EOS>')
    padTokenIndex = wordList.index('<pad>')
    for example in labels:
        eosFound = np.argwhere(example==EOStokenIndex)[0]
        shiftedExample = np.roll(example,1)
        shiftedExample[0] = EOStokenIndex
        # The EOS token was already at the end, so no need for pad
        if (eosFound != (maxLen - 1)):
            shiftedExample[eosFound+1] = padTokenIndex
        laggedLabels.append(shiftedExample)

    # Need to transpose these
    reversedList = np.asarray(reversedList).T.tolist()
    labels = labels.T.tolist()
    laggedLabels = np.asarray(laggedLabels).T.tolist()
    return reversedList, labels, laggedLabels

def translateToSentences(inputs, wList, encoder=False):
    EOStokenIndex = wList.index('<EOS>')
    padTokenIndex = wList.index('<pad>')
    numStrings = len(inputs[0])
    numLengthOfStrings = len(inputs)
    listOfStrings = [''] * numStrings
    for mySet in inputs:
        for index,num in enumerate(mySet):
            if (num != EOStokenIndex and num != padTokenIndex):
                if (encoder):
                    # Encodings are in reverse!
                    listOfStrings[index] = wList[num] + " " + listOfStrings[index]
                else:
                    listOfStrings[index] = listOfStrings[index] + " " + wList[num]
    listOfStrings = [string.strip() for string in listOfStrings]
    return listOfStrings

def getTestInput(inputMessage, wList, maxLen):
    encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
    inputSplit = inputMessage.lower().split()
    for index,word in enumerate(inputSplit):
        try:
            encoderMessage[index] = wList.index(word)
        except ValueError:
            continue
    encoderMessage[index + 1] = wList.index('<EOS>')
    encoderMessage = encoderMessage[::-1]
    encoderMessageList=[]
    for num in encoderMessage:
        encoderMessageList.append([num])
    return encoderMessageList

def intList2Sentence(ids, wList):
    EOStokenIndex = wList.index('<EOS>')
    padTokenIndex = wList.index('<pad>')
    myStr = ""
    listOfResponses=[]
    for num in ids:
        if (num[0] == EOStokenIndex or num[0] == padTokenIndex):
            listOfResponses.append(myStr)
            myStr = ""
        else:
            myStr = myStr + wList[num[0]] + " "
    if myStr:
        listOfResponses.append(myStr)
    listOfResponses = [i for i in listOfResponses if i]
    return listOfResponses



My Version of Model Implementation


In [ ]:
#main model development
#tf.compat.v1.disable_eager_execution()   #tfv2
#disable_eager_execution()
datarange = {'2015-01'}
batchSize = 25
maxEncoderLength = 95                                        #represents max length of input /output sentence
maxDecoderLength = maxEncoderLength
lstmUnits = 112
embeddingDim = lstmUnits
numLayersLSTM = 3
numIterations = 50000
for timeframe in datarange:
    with open(path_to_Shiba+"/wordList.txt","rb") as fp:
        wordList = pickle.load(fp)
    
    wordList.append('<pad>')
    wordList.append('<EOS>')
    vocabSize = len(wordList)
    #question = 'How many dimensions do you want your word vectors to be?: '
    #wordVecDimensions = int(input(question))
    #     if (os.path.isfile('embeddingMatrix.npy')):
    #         wordVectors = np.load('embeddingMatrix.npy')
    #         wordVecDimensions = wordVectors.shape[1]
    
    wordVecDimensions = 5
    padVector = np.zeros((1,wordVecDimensions), dtype = 'int32')
    EOSVector = np.zeros((1,wordVecDimensions), dtype = 'int32')
    generateTrainDataFlag= True
    if(generateTrainDataFlag):
        numTrainingExamples, xTrain, yTrain = createTrainMatrix(path_to_Shiba+"/chatdata/comrepTrain{}.npy".format(timeframe), wordList, maxEncoderLength)
        np.save(path_to_Shiba+'/seq2sexXTrain.npy',xTrain)
        np.save(path_to_Shiba+'/seq2sexYTrain.npy',yTrain)
    else:   
        if (os.path.isfile(path_to_Shiba+'/seq2sexXTrain.npy') and os.path.isfile(path_to_Shiba+'/seq2sexYTrain.npy')):
            xTrain= np.load(path_to_Shiba+'/seq2sexXTrain.npy')
            yTrain= np.load(path_to_Shiba+'/seq2sexYTrain.npy')
            numTrainingExamples = xTrain.shape[0]
        else:
            print("Train Data not found ! please generate train data first.")

    
    print("Training matrix created !")
    
    print("padding index in wordlist",wordList.index('<pad>'),"eos at",wordList.index('<EOS>'))
    tfV2 = """    encoderInputs = [tf.keras.backend.placeholder(dtype=tf.int32, shape=(None,)) for i in range(maxEncoderLength)]
    decoderInputs = [tf.keras.backend.placeholder(dtype=tf.int32, shape=(None,)) for i in range(maxDecoderLength)]
    decoderLabels = [tf.keras.backend.placeholder(dtype=tf.int32, shape=(None,)) for i in range(maxDecoderLength)]
    feedPrevious = tf.keras.backend.placeholder(dtype=tf.bool)
    #long short-term memory
    encoderLstm = tf.compat.v1.nn.rnn_cell.LSTMCell(lstmUnits, state_is_tuple=True)
    encoderLstmKeras = tf.keras.layers.LSTMCell(lstmUnits)
    #     encoderLSTM = tf.nn.rnn_cell.MultiRNNCell([singleCell]*numLayersLSTM, state_is_tuple=True)
    #^ for cloud training 
    decoderOutput, decoderFinalState = tfa.seq2seq.embedding_rnn_seq2seq(encoderInputs, decoderInputs, encoderLstm,vocabSize, vocabSize,embeddingDim,feedPrevious_previous=feedPrevious) 
    
    #PREV:09032021  [tf.nn.seq2seq.embedding_rnn_seq2seq][tf.models.rnn.seq2seq.embedding_rnn_seq2seq]
    #     decoderPrediction = tf.argmax(decoderOutput,2)
    
    #     lossWeights = [tf.ones_like(l,dtype=tf.float32) for l in decoderLabels]
    #     loss = tfa.legacy_seq2seq.sequence_loss(decoderOutput, decoderLabels, lossWeights, vocabSize)
    #     optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
    """    
    #tf v1

    tf.reset_default_graph()
    encoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxEncoderLength)]
    decoderLabels = [tf.placeholder(tf.int32,shape=(None,)) for i in range(maxDecoderLength)]
    decoderInputs = [tf.placeholder(tf.int32,shape=(None,)) for i in range(maxDecoderLength)]
    feedPrevious = tf.placeholder(tf.bool)
    encoderLSTM = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True)
    
    decoderOutputs, decoderFinalState = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(encoderInputs, decoderInputs, encoderLSTM,vocabSize,vocabSize,embedding_size= embeddingDim, feed_previous = feedPrevious)
   
    decoderPrediction = tf.argmax(decoderOutputs,2)
    
    lossWeights = [tf.ones_like(l,dtype= tf.float32) for l in decoderLabels]
    loss = tf.contrib.legacy_seq2seq.sequence_loss(decoderOutputs,decoderLabels,lossWeights,vocabSize)
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)
    sess = tf.Session()
    saver = tf.train.Saver()
    #loading a saved model
    #saver.restore(sess,tf.train.latest_checkpoint('models/'))
    sess.run(tf.global_variables_initializer())
    
    #forwarding result to tensorboard
    tf.summary.scalar('loss',loss)
    merged= tf.summary.merge_all()
    logDir = path_to_Shiba+"/tensorboard/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"/"
    writer = tf.summary.FileWriter(logDir, sess.graph)
     # test strings
    encoderTestStrings = ["Hi","hey ! who are you","hi there","you wanna chill","bob n vagene"]
    zeroVector = np.zeros((1),dtype= 'int32')
    for i in range(numIterations):
        encoderTrain, decodertargerTrain, decoderInputTrain, = getTrainingBatch(xTrain,yTrain, batchSize , maxEncoderLength)
        feedDict = {encoderInputs[t]:encoderTrain[t] for t in range(maxEncoderLength)}
        feedDict.update({decoderLabels[t]:decodertargerTrain[t] for t in range(maxDecoderLength)})
        feedDict.update({decoderInputs[t]:decoderInputTrain[t] for t in range(maxDecoderLength)})
        feedDict.update({feedPrevious:False})
        curLoss ,_,pred = sess.run([loss, optimizer, decoderPrediction],feed_dict = feedDict)
        if(i%50 == 0):
            print("Current Loss",curLoss, "at i = ",i)
            summary = sess.run(merged,feed_dict=feedDict)
            writer.add_summary(summary,i)
        if(i%25==0 and i!=0):
            num = randint(0,len(encoderTestStrings)-1)
            print("Encoder Test String",encoderTestStrings[num])
            inputVector = getTestInput(encoderTestStrings[num],wordList,maxEncoderLength)
            feedDict = {encoderInputs[t]:inputVector[t] for t in range(maxEncoderLength)}
            feedDict.update({decoderLabels[t]:zeroVector for t in range(maxDecoderLength)})
            feedDict.update({decoderInputs[t]:zeroVector for t in range(maxDecoderLength)})
            feedDict.update({feedPrevious: True})
            ids = (sess.run(decoderPrediction,feed_dict= feedDict))
            print(intList2Sentence(ids,wordList))
        if(i%1000 == 0 and i!=0):
            savePath= saver.save(sess,path_to_Shiba+"/models/pretrained_seq2seq.ckpt",global_step=i)
            
            
        
    
    
    


Training matrix created !
padding index in wordlist 8866 eos at 8867
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Current Loss 9.089428 at i =  0
Encoder Test String bob n vagene
['treated, treated, treated, 9/10 ']
Current Loss 8.854749 at i =  50
Encoder Test String hey ! who are you
['9/10 9/10 ']
Encoder Test String bob n vagene
['9/10 9/10 ']
Current Loss 6.7507973 at i =  100
Encoder Test String you wanna chill
[]
Encoder Test String hi there
[]
Current Loss 5.3668413 at i =  150
Encoder Test String hey ! who are you
[]
Encoder Test String bob n vagene
[]
Current Loss 3.8366404 at i =  200
Encoder Test String you wanna chill
[]
Encoder Test String Hi
[]
Current Loss 3.0778

Copypasta Model

In [ ]:

import tensorflow as tf
import numpy as np
import sys
from random import randint
import datetime
from sklearn.utils import shuffle
import pickle
import os
# Removes an annoying Tensorflow warning
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

def createTrainingMatrices(conversationFileName, wList, maxLen):
    conversationDictionary = np.load(conversationFileName,allow_pickle=True).item()
    numExamples = len(conversationDictionary)
    xTrain = np.zeros((numExamples, maxLen), dtype='int32')
    yTrain = np.zeros((numExamples, maxLen), dtype='int32')
    for index,(key,value) in enumerate(conversationDictionary.items()):
        # Will store integerized representation of strings here (initialized as padding)
        encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
        decoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
        # Getting all the individual words in the strings
        keySplit = key.split()
        valueSplit = value.split()
        keyCount = len(keySplit)
        valueCount = len(valueSplit)
        # Throw out sequences that are too long or are empty
        if (keyCount > (maxLen - 1) or valueCount > (maxLen - 1) or valueCount == 0 or keyCount == 0):
            continue
        # Integerize the encoder string
        for keyIndex, word in enumerate(keySplit):
            try:
                encoderMessage[keyIndex] = wList.index(word)
            except ValueError:
                # TODO: This isnt really the right way to handle this scenario
                encoderMessage[keyIndex] = 0
        encoderMessage[keyIndex + 1] = wList.index('<EOS>')
        # Integerize the decoder string
        for valueIndex, word in enumerate(valueSplit):
            try:
                decoderMessage[valueIndex] = wList.index(word)
            except ValueError:
                decoderMessage[valueIndex] = 0
        decoderMessage[valueIndex + 1] = wList.index('<EOS>')
        xTrain[index] = encoderMessage
        yTrain[index] = decoderMessage
    # Remove rows with all zeros
    yTrain = yTrain[~np.all(yTrain == 0, axis=1)]
    xTrain = xTrain[~np.all(xTrain == 0, axis=1)]
    numExamples = xTrain.shape[0]
    return numExamples, xTrain, yTrain

def getTrainingBatch(localXTrain, localYTrain, localBatchSize, maxLen):
    num = randint(0,numTrainingExamples - localBatchSize - 1)
    arr = localXTrain[num:num + localBatchSize]
    labels = localYTrain[num:num + localBatchSize]
    # Reversing the order of encoder string apparently helps as per 2014 paper
    reversedList = list(arr)
    for index,example in enumerate(reversedList):
        reversedList[index] = list(reversed(example))

    # Lagged labels are for the training input into the decoder
    laggedLabels = []
    EOStokenIndex = wordList.index('<EOS>')
    padTokenIndex = wordList.index('<pad>')
    for example in labels:
        eosFound = np.argwhere(example==EOStokenIndex)[0]
        shiftedExample = np.roll(example,1)
        shiftedExample[0] = EOStokenIndex
        # The EOS token was already at the end, so no need for pad
        if (eosFound != (maxLen - 1)):
            shiftedExample[eosFound+1] = padTokenIndex
        laggedLabels.append(shiftedExample)

    # Need to transpose these
    reversedList = np.asarray(reversedList).T.tolist()
    labels = labels.T.tolist()
    laggedLabels = np.asarray(laggedLabels).T.tolist()
    return reversedList, labels, laggedLabels

def translateToSentences(inputs, wList, encoder=False):
    EOStokenIndex = wList.index('<EOS>')
    padTokenIndex = wList.index('<pad>')
    numStrings = len(inputs[0])
    numLengthOfStrings = len(inputs)
    listOfStrings = [''] * numStrings
    for mySet in inputs:
        for index,num in enumerate(mySet):
            if (num != EOStokenIndex and num != padTokenIndex):
                if (encoder):
                    # Encodings are in reverse!
                    listOfStrings[index] = wList[num] + " " + listOfStrings[index]
                else:
                    listOfStrings[index] = listOfStrings[index] + " " + wList[num]
    listOfStrings = [string.strip() for string in listOfStrings]
    return listOfStrings

def getTestInput(inputMessage, wList, maxLen):
    encoderMessage = np.full((maxLen), wList.index('<pad>'), dtype='int32')
    inputSplit = inputMessage.lower().split()
    for index,word in enumerate(inputSplit):
        try:
            encoderMessage[index] = wList.index(word)
        except ValueError:
            continue
    encoderMessage[index + 1] = wList.index('<EOS>')
    encoderMessage = encoderMessage[::-1]
    encoderMessageList=[]
    for num in encoderMessage:
        encoderMessageList.append([num])
    return encoderMessageList

def idsToSentence(ids, wList):
    EOStokenIndex = wList.index('<EOS>')
    padTokenIndex = wList.index('<pad>')
    myStr = ""
    listOfResponses=[]
    for num in ids:
        if (num[0] == EOStokenIndex or num[0] == padTokenIndex):
            listOfResponses.append(myStr)
            myStr = ""
        else:
            myStr = myStr + wList[num[0]] + " "
    if myStr:
        listOfResponses.append(myStr)
    listOfResponses = [i for i in listOfResponses if i]
    return listOfResponses

# Hyperparamters
batchSize = 24
maxEncoderLength = 15
maxDecoderLength = maxEncoderLength
lstmUnits = 112
embeddingDim = lstmUnits
numLayersLSTM = 3
numIterations = 50000

# Loading in all the data structures
with open(path_to_Shiba+"/wordList.txt", "rb") as fp:
    wordList = pickle.load(fp)

vocabSize = len(wordList)

# If you've run the entirety of word2vec.py then these lines will load in
# the embedding matrix.
if (os.path.isfile(path_to_Shiba+'/embeddingMatrix.npy')):
    wordVectors = np.load(path_to_Shiba+'/embeddingMatrix.npy')
    wordVecDimensions = wordVectors.shape[1]
else:
   # question = 'Since we cant find an embedding matrix, how many dimensions do you want your word vectors to be?: '
    wordVecDimensions = 10

# Add two entries to the word vector matrix. One to represent padding tokens,
# and one to represent an end of sentence token
padVector = np.zeros((1, wordVecDimensions), dtype='int32')
EOSVector = np.ones((1, wordVecDimensions), dtype='int32')
if (os.path.isfile(path_to_Shiba+'/embeddingMatrix.npy')):
    wordVectors = np.concatenate((wordVectors,padVector), axis=0)
    wordVectors = np.concatenate((wordVectors,EOSVector), axis=0)

# Need to modify the word list as well
wordList.append('<pad>')
wordList.append('<EOS>')
vocabSize = vocabSize + 2

if (os.path.isfile(path_to_Shiba+'/Seq2SeqXTrain.npy') and os.path.isfile(path_to_Shiba+'/Seq2SeqYTrain.npy')):
    xTrain = np.load(path_to_Shiba+'/Seq2SeqXTrain.npy')
    yTrain = np.load(path_to_Shiba+'/Seq2SeqYTrain.npy')
    print('Finished loading training matrices')
    numTrainingExamples = xTrain.shape[0]
else:
    numTrainingExamples, xTrain, yTrain = createTrainingMatrices(path_to_Shiba+'/chatdata/comrepTrain2015-01.npy', wordList, maxEncoderLength)
    np.save(path_to_Shiba+'/Seq2SeqXTrain.npy', xTrain)
    np.save(path_to_Shiba+'/Seq2SeqYTrain.npy', yTrain)
    print('Finished creating training matrices')

#tf.reset_default_graph()

# Create the placeholders
encoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxEncoderLength)]
decoderLabels = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxDecoderLength)]
decoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxDecoderLength)]
feedPrevious = tf.placeholder(tf.bool)

encoderLSTM = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True)

#encoderLSTM = tf.nn.rnn_cell.MultiRNNCell([singleCell]*numLayersLSTM, state_is_tuple=True)
# Architectural choice of of whether or not to include ^

decoderOutputs, decoderFinalState = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(encoderInputs, decoderInputs, encoderLSTM,
                                                            vocabSize, vocabSize, embeddingDim, feed_previous=feedPrevious)

decoderPrediction = tf.argmax(decoderOutputs, 2)

lossWeights = [tf.ones_like(l, dtype=tf.float32) for l in decoderLabels]
loss = tf.contrib.legacy_seq2seq.sequence_loss(decoderOutputs, decoderLabels, lossWeights, vocabSize)
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

sess = tf.Session()
saver = tf.train.Saver()
# If you're loading in a saved model, uncomment the following line and comment out line 202
#saver.restore(sess, tf.train.latest_checkpoint('models/'))
sess.run(tf.global_variables_initializer())

# Uploading results to Tensorboard
tf.summary.scalar('Loss', loss)
merged = tf.summary.merge_all()
logdir = path_to_Shiba+"/tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

# Some test strings that we'll use as input at intervals during training
encoderTestStrings = ["whats up",
					"hi",
					"hey how are you",
					"what are you up to",
					"that dodgers game was awesome"
					]

zeroVector = np.zeros((1), dtype='int32')

for i in range(numIterations):

    encoderTrain, decoderTargetTrain, decoderInputTrain = getTrainingBatch(xTrain, yTrain, batchSize, maxEncoderLength)
    feedDict = {encoderInputs[t]: encoderTrain[t] for t in range(maxEncoderLength)}
    feedDict.update({decoderLabels[t]: decoderTargetTrain[t] for t in range(maxDecoderLength)})
    feedDict.update({decoderInputs[t]: decoderInputTrain[t] for t in range(maxDecoderLength)})
    feedDict.update({feedPrevious: False})

    curLoss, _, pred = sess.run([loss, optimizer, decoderPrediction], feed_dict=feedDict)

    if (i % 50 == 0):
        print('Current loss:', curLoss, 'at iteration', i)
        summary = sess.run(merged, feed_dict=feedDict)
        writer.add_summary(summary, i)
    if (i % 25 == 0 and i != 0):
        num = randint(0,len(encoderTestStrings) - 1)
        print(encoderTestStrings[num])
        inputVector = getTestInput(encoderTestStrings[num], wordList, maxEncoderLength);
        feedDict = {encoderInputs[t]: inputVector[t] for t in range(maxEncoderLength)}
        feedDict.update({decoderLabels[t]: zeroVector for t in range(maxDecoderLength)})
        feedDict.update({decoderInputs[t]: zeroVector for t in range(maxDecoderLength)})
        feedDict.update({feedPrevious: True})
        ids = (sess.run(decoderPrediction, feed_dict=feedDict))
        print(idsToSentence(ids, wordList))

    if (i % 10000 == 0 and i != 0):
        savePath = saver.save(sess, path_to_Shiba+"/models/pretrained_seq2seq.ckpt", global_step=i)





/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Streaming output truncated to the last 5000 lines.
Instructions for updating:
Use tf.cast instead.
Current loss: 9.090718 at iteration 0
hey how are you
[]
Current loss: 8.776249 at iteration 50
what are you up to
[]
that dodgers game was awesome
[]
Current loss: 7.008749 at iteration 100
what are you up to
[]
that dodgers game was awesome
[]
Current loss: 5.6526217 at iteration 150
that dodgers game was awesome
[]
what are you up to
[]
Current loss: 4.6600633 at iteration 200
what are you up to
[]
hey how are you
[]
Current loss: 4.017374 at iteration 250
hi
[]
that dodgers game was awesome
[]
Current loss: 3.2697093 at iteration 300
that dodgers game was awesome
[]
that dodgers game was awesome
[]
Current loss: 3.2768805 at iteration 350
what are you up to
[]
whats up
[]
Current loss: 3.1650574 at iteration 400
hi
[]
hey how are you
[]
Current loss: 2.983963 at iteration 450
hi
[]
hi
[]
Current loss: 3.288133 at iteration 500
what are you up to
[]
that dodgers game was awesome
[]
Cur

Model Testing !

In [11]:

# Load in hyperparamters
with open(path_to_Shiba+"/chatdata/TrainData/wordList.txt","rb") as fp:
    wordList = pickle.load(fp)

wordList.append('<pad>')
wordList.append('<EOS>')
vocabSize = len(wordList)
# Load in hyperparamters
batchSize = 24
maxEncoderLength = 10
maxDecoderLength = 5
lstmUnits = 112
embeddingDim = lstmUnits
numLayersLSTM = 3

tf.reset_default_graph()
encoderInputs = [tf.placeholder(tf.int32, shape=(None,)) for i in range(maxEncoderLength)]
decoderLabels = [tf.placeholder(tf.int32,shape=(None,)) for i in range(maxDecoderLength)]
decoderInputs = [tf.placeholder(tf.int32,shape=(None,)) for i in range(maxDecoderLength)]
feedPrevious = tf.placeholder(tf.bool)
encoderLSTM = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True)

decoderOutputs, decoderFinalState = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(encoderInputs, decoderInputs, encoderLSTM,vocabSize,vocabSize,embedding_size= embeddingDim, feed_previous = feedPrevious)

decoderPrediction = tf.argmax(decoderOutputs,2)
sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())


# encoderTestStrings = ["Hi","hey ! who are you","hi there","you wanna chill","bob n vagene"]
zeroVector = np.zeros((1),dtype= 'int32')

QString = input(">")
while(True):
    print("Q:",QString)
    if(QString == 'exit' or QString=='quit'):
        break
    inputVector = getTestInput(QString,wordList,maxEncoderLength)
    feedDict = {encoderInputs[t]:inputVector[t] for t in range(maxEncoderLength)}
    feedDict.update({decoderLabels[t]:zeroVector for t in range(maxDecoderLength)})
    feedDict.update({decoderInputs[t]:zeroVector for t in range(maxDecoderLength)})
    feedDict.update({feedPrevious: False})
    ids = (sess.run(decoderPrediction,feed_dict= feedDict))
    
    print("Shiba:",intList2Sentence(ids,wordList))
    QString = input(">")


>bug and error
Q: bug and error
Shiba: ["wondering wondering wondering wondering cheD'ya "]
>quit
Q: quit
